In [30]:
from transformers import *
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [22]:
df = pd.read_csv('data/labeled_data.csv')
data = df[['class', 'tweet']]
data.head()

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [23]:
X_train, X_test, y_train, y_test = train_test_split(data['tweet'], data['class'], random_state=123)
X_train.shape, X_test.shape

((18587,), (6196,))

In [26]:
train_examples = X_train.shape[0]
test_examples = X_test.shape[0]

In [64]:
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [68]:
def convert_to_input_example(X, y):
    df_converted = []
    for i in range(X.shape[0]):
        input_example = InputExample(i, X[i], label=y[i])
        df_converted.append(input_example)
    return df_converted

In [69]:
df_train = convert_to_input_example(X_train, y_train)
df_test = convert_to_input_example(X_test, y_test)

In [24]:
train_dataset = tf.data.Dataset.from_tensor_slices((tf.cast(X_train, tf.string), tf.cast(y_train, tf.int32)))
test_dataset = tf.data.Dataset.from_tensor_slices((tf.cast(X_test, tf.string), tf.cast(y_test, tf.int32)))

In [12]:
config = BertConfig.from_pretrained("bert-base-cased", num_labels=3)
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-cased', config=config)

In [73]:
TASK = "sst-2"
glue_convert_examples_to_features(df_train, tokenizer, max_length=128, task=TASK)

KeyError: 0

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy("accuracy")
model.compile(optimizer=opt, loss=loss, metrics=[metric])

In [9]:
EPOCHS = 10
BATCH_SIZE = 32
EVAL_BATCH_SIZE = BATCH_SIZE * 2
train_steps = train_examples // BATCH_SIZE
valid_steps = valid_examples // EVAL_BATCH_SIZE

history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    steps_per_epoch=train_steps,
    validation_data=valid_dataset,
    validation_steps=valid_steps,
)

NameError: name 'train_examples' is not defined